Import libraries


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import os
os.environ['USE_PYGEOS'] = '0'
from raster2xyz.raster2xyz import Raster2xyz
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import geopandas as gpd
import time

### Import points 
This code snippet loads point data representing tree cover from a shapefile, displays its coordinate system, and visualizes the points.

In [ ]:
print("Points coordinate system")
pointData = gpd.read_file('')
print(pointData.crs)
print("Number of point training samples:", pointData.shape[0])
fig, ax = plt.subplots(figsize=(8, 8))
pointData.plot(ax=ax, markersize=10)
plt.show()


In [ ]:
tif_folder = ''
start = time.process_time()
tif_files = [file for file in os.listdir(tif_folder) if file.endswith('.tif')]
tif_files_sorted = sorted(tif_files)

def get_raster_values(tif_file, coords):
    with rasterio.open(os.path.join(tif_folder, tif_file)) as src:
        values = [x[0] for x in src.sample(coords)]
    return values

for tif_file in tif_files_sorted:
    column_name = os.path.splitext(tif_file)[0]
    coord_list = [(x, y) for x, y in zip(pointData["geometry"].x, pointData["geometry"].y)]
    pointData[column_name] = get_raster_values(tif_file, coord_list)
    print(f"Przetworzono plik: {tif_file}")
print("For the number of traininf samples equal to :",pointData.shape[0] )
print("Procesing time is [s]", time.process_time() - start)
print(pointData.head(6000))
pointData.to_csv('', index=False)


This code reads a CSV file containing multi-temporal SAR data, calculates the Normalized Difference Polarization Index (NDPI) for each date column, and saves the results along with class and geometry information to a new CSV file.

In [ ]:
df = pd.read_csv('')
result_columns = df.columns[3:]  
result_columns = [col for col in result_columns if col.endswith('_VV')] 
result_df = df[['class', 'layer', 'geometry']]
for date_column in result_columns:
    VV = df[date_column]
    VH = df[date_column.replace('_VV', '_VH')]
    # result_df[f'{date_column}_VV_VH'] = VV / VH
    # result_df[f'{date_column}_VH_VV'] = VH / VV
    result_df[f'{date_column}_NDPI'] = (VV - VH) / (VV + VH)
    # result_df[f'{date_column}_POL_Index'] = (VH - VV) / (VH + VV)
    # result_df[f'{date_column}_DPSVI'] = ((((VV.max()) - VV) + VH) / (np.sqrt(2))) * ((VV + VH) / VV) * VH
    # result_df[f'{date_column}_DPSVIm'] = ((VV * VV) + (VV * VH)) / (np.sqrt(2))

result_df.to_csv('', index=False)
